<a href="https://colab.research.google.com/github/Pritam-BA/DAY1/blob/main/DAY_28.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

def create_realistic_hospital_data():
    """
    Create comprehensive hospital management dataset based on Indian healthcare standards
    """
    print("="*60)
    print("CREATING REALISTIC HOSPITAL MANAGEMENT DATASET")
    print("="*60)

    # Hospital configuration (based on typical Indian hospital)
    hospital_config = {
        'total_beds': 250,
        'icu_beds': 30,
        'emergency_beds': 20,
        'general_beds': 200,
        'departments': ['Cardiology', 'Orthopedics', 'General Medicine', 'Pediatrics',
                       'Neurology', 'Emergency', 'ICU', 'Surgery'],
        'shifts': ['Morning', 'Evening', 'Night']
    }

    # Generate 30 days of hospital data
    start_date = datetime(2025, 9, 1)
    days = 30

    # 1. Patient Data
    patients = []
    patient_id = 1000

    for day in range(days):
        current_date = start_date + timedelta(days=day)

        # Daily patient admissions (varies by day of week)
        base_admissions = 15
        if current_date.weekday() == 0:  # Monday higher admissions
            daily_admissions = int(base_admissions * 1.3)
        elif current_date.weekday() >= 5:  # Weekend lower
            daily_admissions = int(base_admissions * 0.7)
        else:
            daily_admissions = base_admissions

        daily_admissions = max(5, daily_admissions + np.random.randint(-3, 4))

        for _ in range(daily_admissions):
            # Patient demographics
            age = max(1, int(np.random.exponential(35)))  # Age distribution
            gender = np.random.choice(['Male', 'Female'], p=[0.55, 0.45])

            # Department assignment
            department = np.random.choice(hospital_config['departments'],
                                        p=[0.20, 0.15, 0.25, 0.10, 0.08, 0.12, 0.05, 0.05])

            # Wait times (realistic for Indian hospitals)
            if department == 'Emergency':
                wait_time = max(5, np.random.exponential(20))  # Emergency faster
            elif department == 'ICU':
                wait_time = max(2, np.random.exponential(10))  # ICU immediate
            else:
                wait_time = max(10, np.random.exponential(45))  # OPD longer waits

            # Length of stay based on department
            if department == 'Emergency':
                length_stay = max(0.5, np.random.exponential(1))
            elif department == 'ICU':
                length_stay = max(1, np.random.exponential(5))
            elif department == 'Surgery':
                length_stay = max(2, np.random.exponential(7))
            else:
                length_stay = max(1, np.random.exponential(3))

            # Treatment cost (in INR)
            base_cost = {'Cardiology': 25000, 'Orthopedics': 35000, 'General Medicine': 8000,
                        'Pediatrics': 12000, 'Neurology': 20000, 'Emergency': 5000,
                        'ICU': 50000, 'Surgery': 40000}

            treatment_cost = base_cost[department] * np.random.uniform(0.6, 1.8)

            # Patient satisfaction (1-5 scale, realistic Indian hospital ratings)
            base_satisfaction = 3.2  # Average Indian hospital rating
            if wait_time < 30:
                satisfaction_bonus = 0.5
            elif wait_time > 60:
                satisfaction_bonus = -0.4
            else:
                satisfaction_bonus = 0

            satisfaction = max(1, min(5, base_satisfaction + satisfaction_bonus + np.random.normal(0, 0.6)))

            # Discharge status
            if current_date.date() < datetime.now().date():
                discharged = np.random.choice([True, False], p=[0.85, 0.15])
            else:
                discharged = False

            patients.append({
                'patient_id': patient_id,
                'admission_date': current_date,
                'age': age,
                'gender': gender,
                'department': department,
                'wait_time_minutes': round(wait_time, 1),
                'length_of_stay_days': round(length_stay, 1),
                'treatment_cost': round(treatment_cost, 2),
                'satisfaction_score': round(satisfaction, 2),
                'discharged': discharged,
                'day_of_week': current_date.strftime('%A')
            })
            patient_id += 1

    # 2. Staff Data
    staff_data = []
    departments = hospital_config['departments']

    for dept in departments:
        # Staff allocation per department
        if dept in ['ICU', 'Emergency']:
            doctors = np.random.randint(8, 12)
            nurses = np.random.randint(15, 25)
        elif dept == 'Surgery':
            doctors = np.random.randint(10, 15)
            nurses = np.random.randint(20, 30)
        else:
            doctors = np.random.randint(5, 8)
            nurses = np.random.randint(10, 15)

        staff_data.append({
            'department': dept,
            'doctors_count': doctors,
            'nurses_count': nurses,
            'total_staff': doctors + nurses,
            'doctor_patient_ratio': round(doctors / 10, 2),  # per 10 patients
            'nurse_patient_ratio': round(nurses / 10, 2)
        })

    # 3. Bed Occupancy Data
    bed_data = []
    for day in range(days):
        current_date = start_date + timedelta(days=day)

        # Current patients in hospital
        current_patients = [p for p in patients
                          if p['admission_date'] <= current_date
                          and not p['discharged']]

        occupied_beds = min(len(current_patients), hospital_config['total_beds'])
        occupancy_rate = (occupied_beds / hospital_config['total_beds']) * 100

        bed_data.append({
            'date': current_date,
            'total_beds': hospital_config['total_beds'],
            'occupied_beds': occupied_beds,
            'available_beds': hospital_config['total_beds'] - occupied_beds,
            'occupancy_rate': round(occupancy_rate, 2),
            'patients_count': len(current_patients)
        })

    # Convert to DataFrames
    patient_df = pd.DataFrame(patients)
    staff_df = pd.DataFrame(staff_data)
    bed_df = pd.DataFrame(bed_data)

    print(f"✅ Dataset Created Successfully!")
    print(f"📊 Patients: {len(patient_df):,}")
    print(f"👥 Staff Records: {len(staff_df)}")
    print(f"🛏️  Bed Tracking Days: {len(bed_df)}")
    print(f"🏥 Total Hospital Beds: {hospital_config['total_beds']}")
    print(f"📈 Average Occupancy: {bed_df['occupancy_rate'].mean():.1f}%")

    return patient_df, staff_df, bed_df, hospital_config

# Create the comprehensive dataset
patient_data, staff_data, bed_data, hospital_info = create_realistic_hospital_data()

# Display sample data
print("\n📊 Sample Patient Data:")
print(patient_data[['patient_id', 'department', 'age', 'wait_time_minutes', 'satisfaction_score', 'treatment_cost']].head())


CREATING REALISTIC HOSPITAL MANAGEMENT DATASET
✅ Dataset Created Successfully!
📊 Patients: 404
👥 Staff Records: 8
🛏️  Bed Tracking Days: 30
🏥 Total Hospital Beds: 250
📈 Average Occupancy: 12.6%

📊 Sample Patient Data:
   patient_id  department  age  wait_time_minutes  satisfaction_score  \
0        1000   Neurology   55               40.9                1.99   
1        1001   Neurology   36              125.5                2.50   
2        1002  Cardiology   33               33.5                3.33   
3        1003   Emergency  143               22.8                3.24   
4        1004  Cardiology   28               11.8                4.38   

   treatment_cost  
0        14399.40  
1        35813.08  
2        16399.97  
3         3079.59  
4        35497.91  


In [2]:
print("\n" + "="*50)
print("HOSPITAL MANAGEMENT DASHBOARD - KEY METRICS")
print("="*50)

# Calculate key dashboard metrics
metrics_dashboard = {
    'patient_satisfaction': patient_data['satisfaction_score'].mean(),
    'bed_occupancy': bed_data['occupancy_rate'].mean(),
    'average_wait_time': patient_data['wait_time_minutes'].mean(),
    'average_treatment_cost': patient_data['treatment_cost'].mean(),
    'total_patients': len(patient_data),
    'average_length_stay': patient_data['length_of_stay_days'].mean(),
    'discharge_rate': (patient_data['discharged'].sum() / len(patient_data)) * 100
}

print("🏥 HOSPITAL PERFORMANCE DASHBOARD")
print("="*40)
for metric, value in metrics_dashboard.items():
    if 'rate' in metric or 'occupancy' in metric or 'discharge' in metric:
        print(f"• {metric.replace('_', ' ').title()}: {value:.1f}%")
    elif 'cost' in metric:
        print(f"• {metric.replace('_', ' ').title()}: ₹{value:,.0f}")
    elif 'satisfaction' in metric:
        print(f"• {metric.replace('_', ' ').title()}: {value:.2f}/5.0")
    elif 'patients' in metric:
        print(f"• {metric.replace('_', ' ').title()}: {int(value):,}")
    else:
        print(f"• {metric.replace('_', ' ').title()}: {value:.1f}")

# Department-wise performance
print(f"\n📋 DEPARTMENT PERFORMANCE:")
dept_performance = patient_data.groupby('department').agg({
    'satisfaction_score': 'mean',
    'wait_time_minutes': 'mean',
    'treatment_cost': 'mean',
    'patient_id': 'count'
}).round(2)
dept_performance.columns = ['Avg_Satisfaction', 'Avg_Wait_Time', 'Avg_Cost', 'Patient_Count']

for dept in dept_performance.index:
    satisfaction = dept_performance.loc[dept, 'Avg_Satisfaction']
    wait_time = dept_performance.loc[dept, 'Avg_Wait_Time']
    count = dept_performance.loc[dept, 'Patient_Count']
    print(f"  {dept}: {satisfaction:.2f}★ satisfaction, {wait_time:.0f}min wait, {count} patients")



HOSPITAL MANAGEMENT DASHBOARD - KEY METRICS
🏥 HOSPITAL PERFORMANCE DASHBOARD
• Patient Satisfaction: 3.39/5.0
• Bed Occupancy: 12.6%
• Average Wait Time: 41.3
• Average Treatment Cost: ₹24,076
• Total Patients: 404
• Average Length Stay: 3.2
• Discharge Rate: 76.5%

📋 DEPARTMENT PERFORMANCE:
  Cardiology: 3.29★ satisfaction, 39min wait, 78 patients
  Emergency: 3.54★ satisfaction, 21min wait, 43 patients
  General Medicine: 3.49★ satisfaction, 43min wait, 109 patients
  ICU: 3.58★ satisfaction, 11min wait, 18 patients
  Neurology: 3.01★ satisfaction, 60min wait, 37 patients
  Orthopedics: 3.22★ satisfaction, 54min wait, 65 patients
  Pediatrics: 3.69★ satisfaction, 36min wait, 36 patients
  Surgery: 3.39★ satisfaction, 45min wait, 18 patients


In [3]:
print("\n" + "="*50)
print("CREATING INTERACTIVE HOSPITAL DASHBOARD")
print("="*50)

# 1. Create Main Dashboard with Subplots
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=('Key Metrics Overview', 'Patient Satisfaction by Department', 'Bed Occupancy Trend',
                   'Wait Time Distribution', 'Daily Patient Admissions', 'Treatment Costs by Department',
                   'Patient Demographics', 'Staff Allocation', 'Performance Trends'),
    specs=[[{"type": "indicator"}, {"type": "bar"}, {"type": "scatter"}],
           [{"type": "histogram"}, {"type": "bar"}, {"type": "box"}],
           [{"type": "pie"}, {"type": "bar"}, {"type": "scatter"}]]
)

# Row 1, Col 1: Key Metrics Indicators
fig.add_trace(
    go.Indicator(
        mode="gauge+number+delta",
        value=metrics_dashboard['bed_occupancy'],
        domain={'x': [0, 1], 'y': [0, 1]},
        title={'text': "Bed Occupancy Rate"},
        delta={'reference': 85},  # Target occupancy
        gauge={
            'axis': {'range': [None, 100]},
            'bar': {'color': "darkblue"},
            'steps': [{'range': [0, 50], 'color': "lightgray"},
                     {'range': [50, 85], 'color': "gray"}],
            'threshold': {'line': {'color': "red", 'width': 4},
                        'thickness': 0.75, 'value': 90}
        }
    ),
    row=1, col=1
)

# Row 1, Col 2: Patient Satisfaction by Department
dept_satisfaction = patient_data.groupby('department')['satisfaction_score'].mean().sort_values(ascending=True)
fig.add_trace(
    go.Bar(x=dept_satisfaction.values, y=dept_satisfaction.index,
           orientation='h', name='Satisfaction'),
    row=1, col=2
)

# Row 1, Col 3: Bed Occupancy Trend
fig.add_trace(
    go.Scatter(x=bed_data['date'], y=bed_data['occupancy_rate'],
               mode='lines+markers', name='Occupancy Rate'),
    row=1, col=3
)

# Row 2, Col 1: Wait Time Distribution
fig.add_trace(
    go.Histogram(x=patient_data['wait_time_minutes'], nbinsx=20, name='Wait Time'),
    row=2, col=1
)

# Row 2, Col 2: Daily Patient Admissions
daily_admissions = patient_data.groupby(patient_data['admission_date'].dt.date).size()
fig.add_trace(
    go.Bar(x=daily_admissions.index, y=daily_admissions.values, name='Daily Admissions'),
    row=2, col=2
)

# Row 2, Col 3: Treatment Costs by Department
fig.add_trace(
    go.Box(x=patient_data['department'], y=patient_data['treatment_cost'], name='Treatment Costs'),
    row=2, col=3
)

# Row 3, Col 1: Patient Demographics (Age Groups)
age_groups = pd.cut(patient_data['age'], bins=[0, 18, 35, 60, 100], labels=['0-18', '19-35', '36-60', '60+'])
age_dist = age_groups.value_counts()
fig.add_trace(
    go.Pie(labels=age_dist.index, values=age_dist.values, name='Age Distribution'),
    row=3, col=1
)

# Row 3, Col 2: Staff Allocation by Department
fig.add_trace(
    go.Bar(x=staff_data['department'], y=staff_data['total_staff'], name='Total Staff'),
    row=3, col=2
)

# Row 3, Col 3: Performance Trends (Satisfaction vs Wait Time)
fig.add_trace(
    go.Scatter(x=patient_data['wait_time_minutes'], y=patient_data['satisfaction_score'],
               mode='markers', name='Satisfaction vs Wait Time', opacity=0.6),
    row=3, col=3
)

# Update layout
fig.update_layout(
    height=1200,
    title_text="🏥 Hospital Management Dashboard - Real-time Analytics",
    title_x=0.5,
    showlegend=False
)

# Show the comprehensive dashboard
fig.show()

print("✅ Interactive Hospital Dashboard Created!")



CREATING INTERACTIVE HOSPITAL DASHBOARD


✅ Interactive Hospital Dashboard Created!
